### Insatlling 'psycopg2-binary'

pip install psycopg2-binary

1. Import libraries

In [18]:
import logging
import requests
import csv
import psycopg2

### Extract 
2. API Calls

In [ ]:
logging.basicConfig(filename='pipeline.log',level = logging.INFO)
logging.info("Script Started")
try:
    url = "https://restcountries.com/v3.1/all?fields=name,region,population"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    logging.info('API Call Successful')
    details = response.json()
except Exception as e:
    logging.error(f"API request failed: {e}")

### Transform
3. Clean the response

In [22]:
clean_details = []
for d in details:
    row = [d["name"]["official"],d['region'],d['population']]
    clean_details.append(row)
clean_details

[['Togolese Republic', 'Africa', 8278737],
 ['Department of Mayotte', 'Africa', 226915],
 ['Georgia', 'Asia', 3714000],
 ['Republic of Vanuatu', 'Oceania', 307150],
 ['Kyrgyz Republic', 'Asia', 6591600],
 ['Republic of Niger', 'Africa', 24206636],
 ["People's Republic of China", 'Asia', 1402112000],
 ['Tuvalu', 'Oceania', 11792],
 ['Union of the Comoros', 'Africa', 869595],
 ['Bosnia and Herzegovina', 'Europe', 3280815],
 ['Kingdom of Bahrain', 'Asia', 1701583],
 ['Federal Republic of Somalia', 'Africa', 15893219],
 ['Collectivity of Saint Barthélemy', 'Americas', 4255],
 ['Republic of Latvia', 'Europe', 1901548],
 ['Cayman Islands', 'Americas', 65720],
 ['Kingdom of the Netherlands', 'Europe', 16655799],
 ['Kingdom of Lesotho', 'Africa', 2142252],
 ['Bolivarian Republic of Venezuela', 'Americas', 28435943],
 ['Republic of Kenya', 'Africa', 53771300],
 ['Republic of Turkey', 'Asia', 84339067],
 ['Republic of Fiji', 'Oceania', 896444],
 ['Republic of Trinidad and Tobago', 'Americas', 13

### Load
3. Load the Data to PostgreSQL

In [27]:
#Connect to server:
conn = psycopg2.connect(
    host = 'localhost',
    database = 'SKET',
    user = 'postgres',
    password = 'shamil'
)
cursor = conn.cursor()

for row in clean_details:
    try:
        cursor.execute('''
            INSERT INTO countries (name, region, population)
            VALUES (%s, %s, %s)
            ON CONFLICT (name) DO NOTHING
        ''',row)
        logging.info('Row updated successfully')
    except Exception as e:
        logging.error(f"Row Failes: {e}")
    
conn.commit()
cursor.close()
conn.close()